## Analizador de projetos Sonar
O analisador for criado para que pudessemos rodar análises do Sonar usando diferentes abordagens como descrito em https://docs.sonarqube.org/latest/analysis/overview/. Os _scanners_ disponíveis são:
* SonarScanner for Gradle
* SonarScanner for MSBuild
* SonarScanner for Maven
* SonarScanner for Jenkins
* SonarScanner for Ant
* SonarScanner

O código foi feito de forma que pudesse ser usado tanto na linha de comando quando como um módulo no _Python_. Para cada um dos item acima, são passados parâmetros diferentes para que análise ocorra com sucesso, além de parâmetros de configuração. Um exemplo de execução seria:
```
python -m auto_sonar -p ../sonar-scanning-examples/sonarqube-scanner-maven/ -s maven -u http://mysonarinstance.com:9000 -t my_secret_token_for_sonar -k my_project_key
```
O comando
```bash
python -m auto_sonar -h
```
pode listar todos as opções disponíveis:
```
usage: auto_sonar [-h] [-p PATH] [-s {msbuild,maven,gradle,ant,sonar_scanner}]
                  [-u URL] [-t TOKEN] [-k PROJECT_KEY]

Auto Analyser for sonar

optional arguments:
  -h, --help            show this help message and exit
  -p PATH, --path PATH
  -s {msbuild,maven,gradle,ant,sonar_scanner}, --scanner {msbuild,maven,gradle,ant,sonar_scanner}
  -u URL, --url URL
  -t TOKEN, --token TOKEN
  -k PROJECT_KEY, --project_key PROJECT_KEY
```

O módulo também pode ser importado no _Python_ da seguinte maneira para ser usado em outro projeto:
```python
# The scanner list is at auto_sonar.SCANNER_LIST
from auto_sonar import AutoSonar

auto_s = AutoSonar(
    project_path='../sonar-scanning-examples/sonarqube-scanner-gradle/',
    scanner='gradle',
    url='...'
    token='...'
    key='...')

auto_s.run()
```

Os requisitos para a análise acontecer são:
* _Python 3.7_ instalado
* Projeto a ser análisado
* Instância do Sonar rodando e pronta para receber análises

## Exploração de dados gerados pela análise de um projeto com o Sonar

Podemos configurar o Sonar para gerar dados em um banco de dados Postgres (por padrão são salvos no H2 Database) para que as análises sejam exploradas usando queries SQL. Uma das formas é executar _containers_ tanto do Postgres quanto do Sonar usando o seguinte `docker-compose.yml`:
```yml
version: '3'

services:
  pg:
    image: postgres
    ports:
    - "5432:5432"
    environment:
      POSTGRES_PASSWORD: sonar
      POSTGRES_USER: sonar
  sonar:
    image: sonarqube
    ports:
    - "9000:9000"
    environment:
      sonar.jdbc.username: sonar
      sonar.jdbc.password: sonar
      sonar.jdbc.url: jdbc:postgresql://pg:5432/sonar
    depends_on:
    - pg
```

Assim, a conexão com o banco estará disponível na porta 5432 e com a instância do Sonar na 9000.


Algumas das tabelas geradas são especialmente importantes quando pensamos em identificar e priorizar dívidas técnicas. A lista abaixo as relaciona com uma explicação concisa de seu conteúdo:
* `rules`
    * Possui todas as regras de qualidade de código que podem se quebradas por alguma dívida técnica, por exemplo: "Strings should not contain new lines".
    * A coluna `name` indica o nome da regra que deve ser respeitada.
* `issues`
    * Relaciona dívidas técnicas de um projeto ou arquivo com uma regra (tabela `rules`) através da coluna `rule_id`.
    * Coluna `severity` indica a gravidade e `message` a descrição e `component_uuid` é uma chave estrangeira para a tabela `projects`.
* `metrics`
    * Contém todas as métricas que podem ser extraídas de um determinado arquivo ou projeto.
    * A coluna `name` contém o nome da métrica e `val_type` o tipo do valor (inteiro, float, etc...).
* `live_metrics`
    * Contém as métricas de arquivos ou projetos.
    * A coluna `metric_id` referencia um registro da tabela `metrics`, `component_uuid` referencia um arquivo e `value` indica o valor numérico da métrica.
* `projects`
    * Todos os arquivos do projeto identificados unicamente pela coluna `uuid`.
    * A coluna `name` contém o nome do arquivo e `scope` indica o tipo do arquivo.

## Isolando métricas de arquivos por dívidas técnicas de determinados tipos
Usando os dados preenchidos pelo Sonar nas tabelas, podemos pensar em relacionar métricas de arquivos com dívidas técnicas presentes neles.
Foram análisadas as métricas `nloc` e `complexity`; com as dívidas do tipo `BLOCKER` e `CRITICAL`. Os projetos escolhidos foram o Maven e o Ant (por enquanto).

Os principais dados dos dois projetos estão sintetizados abaixo:

In [25]:
# Dívidas Técnicas mais presentes e suas métricas para o projeto Ant

count  \
rule_id name                                               metric              
5510    Cognitive Complexity of methods should not be t... ncloc       292.0   
                                                           complexity  292.0   
5370    Handling files is security-sensitive               ncloc       197.0   
                                                           complexity  197.0   
5413    Methods should not be empty                        ncloc       128.0   
                                                           complexity  128.0   
5098    String literals should not be duplicated           ncloc       119.0   
                                                           complexity  119.0   
5297    Dynamically executing code is security-sensitive   ncloc        80.0   
                                                           complexity   80.0   
5298    "clone" should not be overridden                   ncloc        33.0   
                                                           complexity   33.0   
5050    Try-with-resources should be used                  ncloc        29.0   
                                                           complexity   29.0   
5574    Constant names should comply with a naming conv... ncloc        25.0   
                                                           complexity   25.0   

                                                                             mean  \
rule_id name                                               metric                   
5510    Cognitive Complexity of methods should not be t... ncloc       541.845890   
                                                           complexity  130.458904   
5370    Handling files is security-sensitive               ncloc       493.467005   
                                                           complexity  116.771574   
5413    Methods should not be empty                        ncloc       260.648438   
                                                           complexity   57.476562   
5098    String literals should not be duplicated           ncloc       545.529412   
                                                           complexity  129.033613   
5297    Dynamically executing code is security-sensitive   ncloc       297.312500   
                                                           complexity   63.437500   
5298    "clone" should not be overridden                   ncloc       230.666667   
                                                           complexity   60.636364   
5050    Try-with-resources should be used                  ncloc       648.310345   
                                                           complexity  153.068966   
5574    Constant names should comply with a naming conv... ncloc       114.760000   
                                                           complexity   25.240000   

                                                                              std  \
rule_id name                                               metric                   
5510    Cognitive Complexity of methods should not be t... ncloc       404.334276   
                                                           complexity   98.710039   
5370    Handling files is security-sensitive               ncloc       358.039097   
                                                           complexity   91.529596   
5413    Methods should not be empty                        ncloc       269.518310   
                                                           complexity   61.949810   
5098    String literals should not be duplicated           ncloc       395.591722   
                                                           complexity   97.491758   
5297    Dynamically executing code is security-sensitive   ncloc       283.405135   
                                                           complexity   67.151603   
5298    "clone" should not be overridden                   ncloc       153.7035

In [37]:
# Dívidas Técnicas mais presentes e suas métricas para o projeto Maven

count  \
rule_id name                                               metric              
5510    Cognitive Complexity of methods should not be t... ncloc       110.0   
                                                           complexity  110.0   
5413    Methods should not be empty                        ncloc        93.0   
                                                           complexity   93.0   
5370    Handling files is security-sensitive               ncloc        79.0   
                                                           complexity   79.0   
5098    String literals should not be duplicated           ncloc        72.0   
                                                           complexity   72.0   
5290    Constants should not be defined in interfaces      ncloc        46.0   
                                                           complexity   46.0   
5574    Constant names should comply with a naming conv... ncloc        37.0   
                                                           complexity   37.0   
5205    Fields in a "Serializable" class should either ... ncloc        33.0   
                                                           complexity   33.0   
5380    Using regular expressions is security-sensitive    ncloc        12.0   
                                                           complexity   12.0   

                                                                             mean  \
rule_id name                                               metric                   
5510    Cognitive Complexity of methods should not be t... ncloc       520.745455   
                                                           complexity   90.309091   
5413    Methods should not be empty                        ncloc       211.731183   
                                                           complexity   52.956989   
5370    Handling files is security-sensitive               ncloc       509.822785   
                                                           complexity   84.974684   
5098    String literals should not be duplicated           ncloc       723.291667   
                                                           complexity  148.277778   
5290    Constants should not be defined in interfaces      ncloc        36.108696   
                                                           complexity    3.282609   
5574    Constant names should comply with a naming conv... ncloc        35.108108   
                                                           complexity    3.054054   
5205    Fields in a "Serializable" class should either ... ncloc        97.575758   
                                                           complexity   16.333333   
5380    Using regular expressions is security-sensitive    ncloc       335.500000   
                                                           complexity   57.666667   

                                                                              std  \
rule_id name                                               metric                   
5510    Cognitive Complexity of methods should not be t... ncloc       379.616041   
                                                           complexity   72.819463   
5413    Methods should not be empty                        ncloc       205.521930   
                                                           complexity   41.182762   
5370    Handling files is security-sensitive               ncloc       455.047187   
                                                           complexity   81.440237   
5098    String literals should not be duplicated           ncloc       781.323659   
                                                           complexity  195.605180   
5290    Constants should not be defined in interfaces      ncloc        83.966720   
                                                           complexity   20.658776   
5574    Constant names should comply with a naming conv... ncloc        19.4118

Também podemos obter as metricas de todos os arquivos do projeto para termos uma comparação:

* Ant
    * `complexity`
        * count    923.000000
        * mean      27.565547
        * std       43.122798
        * min        0.000000
        * 25%        4.000000
        * 50%       13.000000
        * 75%       31.000000
        * max      401.000000
    * `nloc`
        * count     925.000000
        * mean      121.480000
        * std       176.656522
        * min         1.000000
        * 25%        24.000000
        * 50%        61.000000
        * 75%       134.000000
        * max      1667.000000 
* Maven
    * `complexity`
        * count    700.000000
        * mean      14.334286
        * std       35.735593
        * min        0.000000
        * 25%        0.000000
        * 50%        5.000000
        * 75%       15.000000
        * max      664.000000
    * `nloc`
        * count     715.000000
        * mean       88.925874
        * std       172.153021
        * min         1.000000
        * 25%        14.500000
        * 50%        40.000000
        * 75%        95.000000
        * max      2693.000000